In [ ]:
# ✅ Install dependencies if needed
!pip install sqlalchemy psycopg2-binary

In [ ]:
# 🔌 Connect to PostgreSQL
from sqlalchemy import create_engine, text

# 👉 Edit these for your environment
DB_USER = "postgres"
DB_PASS = "your_password"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "ecoimpact"

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [ ]:
# 🧼 Load and execute cleaned SQL dump
sql_path = "ecoimpact_dump.sql"

with open(sql_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

valid_sql = []
in_copy_block = False

for line in lines:
    stripped = line.strip()
    if stripped.lower().startswith("copy ") and "from stdin" in stripped.lower():
        in_copy_block = True
        continue
    if stripped == "\\.":
        in_copy_block = False
        continue
    if in_copy_block:
        continue
    if (
        not stripped
        or stripped.startswith("--")
        or any(stripped.startswith(prefix) for prefix in ("Name:", "Type:", "Schema:", "Owner:"))
    ):
        continue
    valid_sql.append(line)

sql_cleaned = "".join(valid_sql)

with engine.connect() as conn:
    for statement in sql_cleaned.split(";"):
        stmt = statement.strip()
        if stmt:
            try:
                conn.execute(text(stmt + ";"))
            except Exception as e:
                print(f"⚠️ Skipped: {e}")
    conn.commit()

print("✅ Cleaned SQL schema loaded!")

In [ ]:
# ✅ Verify tables
import pandas as pd

tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema='public';", engine)
tables